<a href="https://colab.research.google.com/github/YinGuoX/Deep_Learning_Keras_WithDeeplizard/blob/master/18_Process_Images_For_Fine_Tuned_MobileNet_With_TensorFlow's_Keras_API.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Process Images For Fine-Tuned MobileNet With TensorFlow's Keras API
在本集中，我们将基于对MobileNet的了解来准备和处理自己的手语数字定制图像数据集。 这些图像将在以后用于训练我们经过微调的MobileNet模型。

## 1. 获取数据
我们现在使用的新数据集没有包含在ImageNet中的类，也不是非常类似于MobileNet最初训练的ImageNet类。稍后，我们将一起完成调整模型的过程，以根据新数据进行预测。

这个数据集包含了手语数字的图像。共有10个类别，分别标为0到9，每个类别都由手的图像组成，这些图像表示特定数字的符号。

每个类别都有204到208个样本。 总数据集包含2062个样本。

该数据集在[Kaggle](https://www.kaggle.com/ardamavi/sign-language-digits-dataset)上可以用作灰度图像，在[Github](https://github.com/ardamavi/Sign-Language-Digits-Dataset)上也可以用作RGB图像。 我们将使用RGB数据集。


## 2. 组织数据
在我们开始调优模型之前，我们首先需要组织磁盘上的图像。它们将以我们在前几集中使用的猫和狗数据集相同的方式进行组织。

通过代码，我们将创建train、validation和test集的目录，将图像组织到磁盘上它们各自的0到9的类中，然后洗牌数据集并将图像分成train、validation和test集。

首先，下载数据并保存到your-current-nb-directory / data / Sign-Language-Digits-Dataset。 提取标记为0到9的子目录，并将这些目录直接嵌套在Sign-Language-Digits-Dataset目录下。

**注意：**前两行代码将目录更改为存储数据的相关目录，然后检查以确保目录结构尚未设置。


In [ ]:
# 在colab上使用的云端数据
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
path = "/content/drive/My Drive/DeepLearning_AI_Course/Deeplizard/Keras - Python Deep Learning Neural Network API/Data/Sign-Language-Digits-Dataset"

# 更改当前系统的路径
os.chdir(path)
# 显示当前路径的文件夹名字
print(os.listdir())

# 获得当前路径
os.getcwd()

['3', '9', '5', '1', '2', '0', '8', '4', '6', '7', 'valid', 'test']


'/content/drive/My Drive/DeepLearning_AI_Course/Deeplizard/Keras - Python Deep Learning Neural Network API/Data/Sign-Language-Digits-Dataset'

In [ ]:
import os
import shutil
import random

In [ ]:
if os.path.isdir('train/0/') is False:
  os.mkdir('train')
  for i in range(0,10):
    shutil.move('{}'.format(i),'train')
    
print(os.listdir())

['valid', 'test', 'train']


In [ ]:
for i in range(0, 10):
    os.mkdir(f'valid/{i}')
    os.mkdir(f'test/{i}')

    valid_samples = random.sample(os.listdir(f'train/{i}'), 30)
    for j in valid_samples:
        shutil.move(f'train/{i}/{j}', f'valid/{i}')

    test_samples = random.sample(os.listdir(f'train/{i}'), 5)
    for k in test_samples:
        shutil.move(f'train/{i}/{k}', f'test/{i}')

## 3.处理数据
在将图像数据全部组织在磁盘上之后，我们需要以与之前使用的cat和dog数据集完全相同的方式，为训练，验证和测试集创建字典迭代器。

In [ ]:
os.chdir("..")
print(os.listdir())

['dogs-vs-cats', 'MobileNet-samples', 'Sign-Language-Digits-Dataset']


In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dense, Flatten, BatchNormalization, Conv2D, MaxPool2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import confusion_matrix
import itertools
import os
import shutil
import random
import glob
import matplotlib.pyplot as plt
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
%matplotlib inline

In [ ]:

train_path = './Sign-Language-Digits-Dataset/train'
valid_path = './Sign-Language-Digits-Dataset/valid'
test_path = './Sign-Language-Digits-Dataset/test'
train_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.mobilenet.preprocess_input).flow_from_directory(
    directory=train_path, target_size=(224,224), batch_size=10)
valid_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.mobilenet.preprocess_input).flow_from_directory(
    directory=valid_path, target_size=(224,224), batch_size=10)
test_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.mobilenet.preprocess_input).flow_from_directory(
    directory=test_path, target_size=(224,224), batch_size=10, shuffle=False)

Found 1712 images belonging to 10 classes.
Found 300 images belonging to 10 classes.
Found 50 images belonging to 10 classes.


**注意:**我们提供给ImageDataGenerator的preprocessing_function参数。 我们将此设置为等于tf.keras.applications.mobilenet.preprocess_input。 这将对从flow_from_directory（）获得的图像（先前解释）进行必要的MobileNet预处理。

对于test_batches，我们还提供了一个附加参数shuffle = False，这将导致不对测试数据集进行混洗，以便稍后可以访问相应的未混洗的测试标签以绘制成混淆矩阵。

现在已经准备好数据并进行处理。 接下来，我们将继续对模型进行微调。